# Functions

In [642]:
def print_shape_of_dataframes(**kwargs):
    """
    Print the shape (number of rows and columns) of multiple pandas DataFrame
    objects.

    Parameters:
    -------------
    **kwargs: dict
        Keyword arguments where the keys are the names of the DataFrame
        objects (as string) and the values are the DataFrame objects
        themselves.

    Notes:
    ------------
    The functon dynamically handles any number of DataFrame objects passed
    as keyword arguments.
    Ensure the passed objects are valid pandas DataFrame to avoid unexpected
    errors.
    """
    for name, value in kwargs.items():
        if hasattr(value, 'shape'):
            print(f'DataFrame \033[94m{name.title()}\033[0m has {value.shape[0]} rows and {value.shape[1]} columns.')
            print('------------------------------------')
        else:
            raise ValueError(f'The object 033[94m{name.title()}\033[0m is not a valid DataFrame.')

In [764]:
from collections import Counter

def delete_outliers(data_frame: pd.DataFrame, n: int, features: list) -> list:
    """
    Detect and Identifies outliers in the specified features of a 
    DataFrame.

    This function iterates through a list of numerical features and detects
    outliers in each feature based on the IQR method. An outlier is defined
    as a value below Q1 - 1.5 * IQR or above Q3 + 1.5 * IQR.
    If an index appears as an outlier in more than N features, it is added to the 
    final list of multiple outliers.

    Parameters:
    ---------------
    data_frame: pd.DataFrame
        The DataFrame containing the data to analyze.

    n: int
        The minimum number of features in which a data point must be an outlier
        to be considered a multiple outlier.

    features: list
        A list of column names (features) to analyze for outliers.
        Those should be numerical columns.

    Returns:
    -------------
    list
        A list of indices corresponding to data points that are outliers
        in more than n features.

    Raises:
    ------------
    TypeError
        If data_frame is not a pandas DataFrame, or if features is not a list.

    ValueError
        If n is negative, or if a feature in the features list is not in
        the DataFrame.
    """
    if not isinstance(data_frame, pd.DataFrame):
        raise TypeError("`data_frame` must be a pandas DataFrame.")
    if not isinstance(features, list):
        raise TypeError("`features` must be a list of column names.")
    if not isinstance(n, int) or n < 0:
        raise ValueError("`n` must be a non-negative integer.")
        
    outliers = []
    for feature in features:
        Q1 = np.percentile(data_frame[feature], 25)
        Q3 = np.percentile(data_frame[feature], 75)
        IQR = Q3 - Q1
        lower = Q1 - 1.5*IQR
        upper = Q3 + 1.5*IQR

        outlier_list = data_frame[(data_frame[feature] < lower) | (data_frame[feature] > upper)].index
        outliers.extend(outlier_list)
    outliers = Counter(outliers)
    multiple_outliers = list(key for key, value in outliers.items() if value > n)
    return multiple_outliers



# 1. Import Libraries

In [643]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns

import missingno
import re

import datetime

# 2. Import and Read data

In [644]:
df = pd.read_csv('Uncleaned_DS_jobs.csv')
df.head()

,index,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"
1,1,Data Scientist,$137K-$171K (Glassdoor est.),"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech\n4.2,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,1968,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),-1
2,2,Data Scientist,$137K-$171K (Glassdoor est.),Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group\n3.8,"Boston, MA","Boston, MA",1001 to 5000 employees,1981,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),-1
3,3,Data Scientist,$137K-$171K (Glassdoor est.),JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON\n3.5,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,2000,Company - Public,Electrical & Electronic Manufacturing,Manufacturing,$100 to $500 million (USD),"MKS Instruments, Pfeiffer Vacuum, Agilent Tech..."
4,4,Data Scientist,$137K-$171K (Glassdoor est.),Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions\n2.9,"New York, NY","New York, NY",51 to 200 employees,1998,Company - Private,Advertising & Marketing,Business Services,Unknown / Non-Applicable,"Commerce Signals, Cardlytics, Yodlee"


In [645]:
df_cleaned = pd.read_csv('Cleaned_DS_Jobs.csv')
df_cleaned.head()

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Type of ownership,Industry,...,company_age,python,excel,hadoop,spark,aws,tableau,big_data,job_simp,seniority
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,"New York, NY","New York, NY",1001 to 5000 employees,Nonprofit Organization,Insurance Carriers,...,27,0,0,0,0,1,0,0,data scientist,senior
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,"Chantilly, VA","Herndon, VA",5001 to 10000 employees,Company - Public,Research & Development,...,52,0,0,1,0,0,0,1,data scientist,na
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,"Boston, MA","Boston, MA",1001 to 5000 employees,Private Practice / Firm,Consulting,...,39,1,1,0,0,1,0,0,data scientist,na
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,"Newton, MA","Bad Ragaz, Switzerland",501 to 1000 employees,Company - Public,Electrical & Electronic Manufacturing,...,20,1,1,0,0,1,0,0,data scientist,na
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,"New York, NY","New York, NY",51 to 200 employees,Company - Private,Advertising & Marketing,...,22,1,1,0,0,0,0,0,data scientist,na


In [646]:
print_shape_of_dataframes(uncleaned_df=df, cleaned_df=df_cleaned)

DataFrame Uncleaned_Df has 672 rows and 15 columns.
------------------------------------
DataFrame Cleaned_Df has 660 rows and 27 columns.
------------------------------------


The raw dataset (Uncleaned_DS_jobs.csv) was imported and initially contained 672 rows and 15 columns, representing the unprocessed state of the data with potential inconsistencies, missing values, and redundant information. After cleaning, the dataset (Cleaned_DS_Jobs.csv) was reduced to 660 rows and expanded to 27 columns, indicating the removal of invalid or unnecessary records and the addition of new features or transformations during the cleaning process. This transformation resulted in a more structured and reliable dataset, ready for further analysis or modeling. The changes reflect the efforts to improve data quality and usability.

# 3. Data Description and Transformation

In [647]:
df.shape

(672, 15)

In [648]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 672 entries, 0 to 671
Data columns (total 15 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   index              672 non-null    int64  
 1   Job Title          672 non-null    object 
 2   Salary Estimate    672 non-null    object 
 3   Job Description    672 non-null    object 
 4   Rating             672 non-null    float64
 5   Company Name       672 non-null    object 
 6   Location           672 non-null    object 
 7   Headquarters       672 non-null    object 
 8   Size               672 non-null    object 
 9   Founded            672 non-null    int64  
 10  Type of ownership  672 non-null    object 
 11  Industry           672 non-null    object 
 12  Sector             672 non-null    object 
 13  Revenue            672 non-null    object 
 14  Competitors        672 non-null    object 
dtypes: float64(1), int64(2), object(12)
memory usage: 78.9+ KB


The uncleaned dataset consists of 672 entries and 15 columns, with a mix of numeric and object data types, including job titles, salary estimates, company details, and descriptive fields. All columns have non-null values, suggesting no missing data, but further inspection is needed to identify potential inconsistencies or formatting issues within the fields.

In [649]:
df.columns

Index(['index', 'Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

## `Index Feature`

The index column in the uncleaned dataset contains sequential integer values from 0 to 671, which do not provide meaningful information for analysis. Therefore, it is considered redundant and should be removed before proceeding with data cleaning.

In [650]:
df['index'].unique()

array([  0,   1,   2,   3,   4,   5,   6,   7,   8,   9,  10,  11,  12,
        13,  14,  15,  16,  17,  18,  19,  20,  21,  22,  23,  24,  25,
        26,  27,  28,  29,  30,  31,  32,  33,  34,  35,  36,  37,  38,
        39,  40,  41,  42,  43,  44,  45,  46,  47,  48,  49,  50,  51,
        52,  53,  54,  55,  56,  57,  58,  59,  60,  61,  62,  63,  64,
        65,  66,  67,  68,  69,  70,  71,  72,  73,  74,  75,  76,  77,
        78,  79,  80,  81,  82,  83,  84,  85,  86,  87,  88,  89,  90,
        91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101, 102, 103,
       104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116,
       117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129,
       130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142,
       143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155,
       156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168,
       169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 18

In [651]:
cleaning_df = df.drop(labels='index', axis=1)

In [652]:
cleaning_df.head(1)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,Size,Founded,Type of ownership,Industry,Sector,Revenue,Competitors
0,Sr Data Scientist,$137K-$171K (Glassdoor est.),Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst\n3.1,"New York, NY","New York, NY",1001 to 5000 employees,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna"


## `Job Title Feature`

The Job Title column contains various job titles related to data science and other technical roles, including variations such as "Data Scientist," "Data Engineer," "Machine Learning Engineer," and more. Given the diversity and specificity of these titles, we have decided to leave this column as is, without applying any transformations. It is important to retain these distinct titles as they represent unique job roles and are crucial for any analysis regarding job market trends, skill requirements, or salary estimation.

In [653]:
cleaning_df['Job Title'].unique()

array(['Sr Data Scientist', 'Data Scientist',
       'Data Scientist / Machine Learning Expert',
       'Staff Data Scientist - Analytics',
       'Data Scientist - Statistics, Early Career', 'Data Modeler',
       'Experienced Data Scientist', 'Data Scientist - Contract',
       'Data Analyst II', 'Medical Lab Scientist',
       'Data Scientist/Machine Learning', 'Human Factors Scientist',
       'Business Intelligence Analyst I- Data Insights',
       'Data Scientist - Risk', 'Data Scientist-Human Resources',
       'Senior Research Statistician- Data Scientist', 'Data Engineer',
       'Associate Data Scientist', 'Business Intelligence Analyst',
       'Senior Analyst/Data Scientist', 'Data Analyst',
       'Machine Learning Engineer', 'Data Analyst I',
       'Scientist - Molecular Biology',
       'Computational Scientist, Machine Learning',
       'Senior Data Scientist', 'Jr. Data Engineer',
       'E-Commerce Data Analyst', 'Data Analytics Engineer',
       'Product Data Scient

## `Salary Estimate Feature`

The Salary Estimate column originally contained salary ranges in a string format, including values such as $137K-$171K (Glassdoor est.) and $75K-$131K (Glassdoor est.). To standardize the data and make it more useful for analysis, we cleaned this column by extracting the salary range and removing the text and currency symbols.

We used a regular expression to capture the numeric values within the salary range, converting them into a simplified format (e.g., 137-171 for $137K-$171K). This cleaned format provides a consistent range of values for each entry.

In [654]:
cleaning_df['Salary Estimate'].unique()

array(['$137K-$171K (Glassdoor est.)', '$75K-$131K (Glassdoor est.)',
       '$79K-$131K (Glassdoor est.)', '$99K-$132K (Glassdoor est.)',
       '$90K-$109K (Glassdoor est.)', '$101K-$165K (Glassdoor est.)',
       '$56K-$97K (Glassdoor est.)', '$79K-$106K (Glassdoor est.)',
       '$71K-$123K (Glassdoor est.)', '$90K-$124K (Glassdoor est.)',
       '$91K-$150K (Glassdoor est.)', '$141K-$225K (Glassdoor est.)',
       '$145K-$225K(Employer est.)', '$79K-$147K (Glassdoor est.)',
       '$122K-$146K (Glassdoor est.)', '$112K-$116K (Glassdoor est.)',
       '$110K-$163K (Glassdoor est.)', '$124K-$198K (Glassdoor est.)',
       '$79K-$133K (Glassdoor est.)', '$69K-$116K (Glassdoor est.)',
       '$31K-$56K (Glassdoor est.)', '$95K-$119K (Glassdoor est.)',
       '$212K-$331K (Glassdoor est.)', '$66K-$112K (Glassdoor est.)',
       '$128K-$201K (Glassdoor est.)', '$138K-$158K (Glassdoor est.)',
       '$80K-$132K (Glassdoor est.)', '$87K-$141K (Glassdoor est.)',
       '$92K-$155K (Glassdo

In [655]:
def convert_salary_estimate_feature(value):
    """
    Converts a salary range from the format '$XK-$YK (Glassdoor est.)' to a simplified 
    format 'X-Y' by extracting the lower and upper bounds of the salary range.

    Parameters:
    ---------------
    value (str): 
        A string representing the salary range, e.g., '$137K-$171K (Glassdoor est.)'.

    Returns:
    ---------------
    str: 
        A string representing the simplified salary range, e.g., '137-171'.
    """
    match = re.search(r'\$(\d+)(?:K)-\$(\d+)(?:K)', value)
    return f'{match.group(1)}-{match.group(2)}'

In [656]:
cleaning_df['Salary Estimate'] = cleaning_df['Salary Estimate'].apply(convert_salary_estimate_feature)

In [657]:
cleaning_df['Salary Estimate'].unique()

array(['137-171', '75-131', '79-131', '99-132', '90-109', '101-165',
       '56-97', '79-106', '71-123', '90-124', '91-150', '141-225',
       '145-225', '79-147', '122-146', '112-116', '110-163', '124-198',
       '79-133', '69-116', '31-56', '95-119', '212-331', '66-112',
       '128-201', '138-158', '80-132', '87-141', '92-155', '105-167'],
      dtype=object)

## `Job Description Feature`

The Job Description feature contains detailed textual descriptions of job roles, responsibilities, qualifications, and other relevant information for each job posting. These descriptions vary widely in content, often including specific technical skills, required qualifications, and company values. The goal is to provide a comprehensive overview of the job, the company, and the expectations for the role.

Given the highly variable nature of these descriptions (such as detailed healthcare analytics roles, IT security roles, or consulting positions), no cleaning or modifications were made to this feature. The raw text was retained for further analysis as needed.

In [658]:
cleaning_df['Job Description'].unique()[:3]

array(["Description\n\nThe Senior Data Scientist is responsible for defining, building, and improving statistical models to improve business processes and outcomes in one or more healthcare domains such as Clinical, Enrollment, Claims, and Finance. As part of the broader analytics team, Data Scientist will gather and analyze data to solve and address complex business problems and evaluate scenarios to make predictions on future outcomes and work with the business to communicate and support decision-making. This position requires strong analytical skills and experience in analytic methods including multivariate regressions, hierarchical linear models, regression trees, clustering methods and other complex statistical techniques.\n\nDuties & Responsibilities:\n\n• Develops advanced statistical models to predict, quantify or forecast various operational and performance metrics in multiple healthcare domains\n• Investigates, recommends, and initiates acquisition of new data resources from 

## `Rating Feature`

The Rating feature contains values where negative ratings were present, which are handled by the cleaning_ratings function. This function checks if the rating is below zero and converts it to 0, while leaving all other ratings unchanged. After applying this cleaning process, the ratings are updated to reflect non-negative values only.

In [659]:
cleaning_df['Rating'].unique()

array([ 3.1,  4.2,  3.8,  3.5,  2.9,  3.9,  4.4,  3.6,  4.5,  4.7,  3.7,
        3.4,  4.1,  3.2,  4.3,  2.8,  5. ,  4.8,  3.3,  2.7,  2.2,  2.6,
        4. ,  2.5,  4.9,  2.4, -1. ,  2.3,  4.6,  3. ,  2.1,  2. ])

In [660]:
cleaning_df['Rating'].dtype

dtype('float64')

In [661]:
def cleaning_ratings(value):
    """
    Cleans the rating values by replacing any negative ratings with 0, 
    while leaving positive ratings unchanged.

    Parameters:
    -------------
    value (float): 
        The rating value to be cleaned. It can be a positive float, zero, or a negative value.

    Returns:
    -------------
    float: 
        A cleaned rating value. Negative values are replaced with 0, while other values remain the same.
    """
    if value < 0:
        return 0
    else:
        return value

In [662]:
cleaning_df['Rating'] = cleaning_df['Rating'].apply(cleaning_ratings)

In [663]:
cleaning_df['Rating'].unique()

array([3.1, 4.2, 3.8, 3.5, 2.9, 3.9, 4.4, 3.6, 4.5, 4.7, 3.7, 3.4, 4.1,
       3.2, 4.3, 2.8, 5. , 4.8, 3.3, 2.7, 2.2, 2.6, 4. , 2.5, 4.9, 2.4,
       0. , 2.3, 4.6, 3. , 2.1, 2. ])

In [664]:
cleaning_df[cleaning_df['Rating'] == 0].index

Index([154, 158, 230, 282, 285, 290, 319, 322, 329, 338, 351, 357, 358, 359,
       360, 361, 362, 388, 389, 409, 411, 425, 430, 431, 437, 438, 440, 457,
       459, 495, 496, 497, 498, 499, 500, 504, 519, 524, 555, 568, 613, 615,
       637, 650, 656, 657, 660, 664, 668, 669],
      dtype='int64')

In [665]:
cleaning_df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors'],
      dtype='object')

## `Company Name Feature`

The Company Name feature originally contained both company names and ratings separated by a newline character (\n). The clean_company_name_feature function cleans this feature by removing the newline character and any text following it. After applying this transformation, the Company Name column now contains only the company names, without any extraneous information.

In [666]:
cleaning_df['Company Name'].unique()

array(['Healthfirst\n3.1', 'ManTech\n4.2', 'Analysis Group\n3.8',
       'INFICON\n3.5', 'Affinity Solutions\n2.9', 'HG Insights\n4.2',
       'Novartis\n3.9', 'iRobot\n3.5', 'Intuit - Data\n4.4',
       'XSELL Technologies\n3.6', 'Novetta\n4.5', '1904labs\n4.7',
       'PNNL\n3.7', 'Old World Industries\n3.1',
       'Mathematica Policy Research\n3.4',
       'Guzman & Griffin Technologies (GGTI)\n4.4',
       'Upside Business Travel\n4.1', 'Buckman\n3.5',
       'Insight Enterprises, Inc.\n4.2', 'Tower Health\n3.5',
       'Triplebyte\n3.2', 'PulsePoint\n4.3', 'Exponent\n3.5',
       'Guardian Life\n3.5',
       'Spectrum Communications and Consulting\n3.4',
       'Oversight Systems\n4.7', 'LSQ\n4.2',
       'MIT Lincoln Laboratory\n3.8', 'Kingfisher Systems\n4.5',
       'Formation\n2.8', 'Cohere Health\n5.0', 'Acuity Insurance\n4.8',
       'Chef\n3.6', 'Puget Sound Energy\n3.3', 'Sandhills Global\n2.7',
       'A Place for Mom\n2.7', 'Great-Circle Technologies\n2.2',
       'Edmu

In [667]:
def clean_company_name_feature(value):
    """
    Cleans the input string by removing the newline character ('\n') and any text that follows it.

    Parameters:
    -------------
    value (str): 
        A string containing company names and additional information after a newline character.

    Returns:
    -------------
    str: 
        A cleaned version of the input string with the part after the newline removed.
    """
    return re.sub(r'\n.*', '', value)

In [668]:
cleaning_df['Company Name'] = cleaning_df['Company Name'].apply(clean_company_name_feature)

In [669]:
cleaning_df['Company Name'].unique()

array(['Healthfirst', 'ManTech', 'Analysis Group', 'INFICON',
       'Affinity Solutions', 'HG Insights', 'Novartis', 'iRobot',
       'Intuit - Data', 'XSELL Technologies', 'Novetta', '1904labs',
       'PNNL', 'Old World Industries', 'Mathematica Policy Research',
       'Guzman & Griffin Technologies (GGTI)', 'Upside Business Travel',
       'Buckman', 'Insight Enterprises, Inc.', 'Tower Health',
       'Triplebyte', 'PulsePoint', 'Exponent', 'Guardian Life',
       'Spectrum Communications and Consulting', 'Oversight Systems',
       'LSQ', 'MIT Lincoln Laboratory', 'Kingfisher Systems', 'Formation',
       'Cohere Health', 'Acuity Insurance', 'Chef', 'Puget Sound Energy',
       'Sandhills Global', 'A Place for Mom', 'Great-Circle Technologies',
       'Edmunds.com', 'Cambridge Associates, LLC',
       'Liberty Mutual Insurance', 'Cenlar', 'Arsenal Biosciences',
       'Eversight', 'Pfizer', 'Klaviyo', 'Intellectual Ventures',
       'GovTech', 'Quick Base', 'Giving Assistant', 'T

## `Location Feature`

The Location feature in the dataset initially contained location information in the form of city names, states, and sometimes additional details. To standardize this, two functions were applied. The job_location_new_feature function extracts the state abbreviation from location strings by checking whether they contain a city-state format (separated by a comma) or are just a state name, returning the state abbreviation in uppercase. The clean_location_feature function further refines the location by keeping only the state information, removing any city names. After applying these transformations, a new column, Location, is generated, containing the state names in a standardized format, with values such as 'NY', 'CA', 'TX', and 'FL', representing state abbreviations, or specific locations like 'Remote'. This ensures consistency and prepares the data for further analysis.

In [670]:
cleaning_df['Location'].unique()

array(['New York, NY', 'Chantilly, VA', 'Boston, MA', 'Newton, MA',
       'Santa Barbara, CA', 'Cambridge, MA', 'Bedford, MA',
       'San Diego, CA', 'Chicago, IL', 'Herndon, VA', 'Saint Louis, MO',
       'Richland, WA', 'Northbrook, IL', 'Washington, DC', 'Remote',
       'Memphis, TN', 'Plano, TX', 'West Grove, PA', 'Phoenix, AZ',
       'Appleton, WI', 'Atlanta, GA', 'Orlando, FL', 'Lexington, MA',
       'McLean, VA', 'San Francisco, CA', 'Sheboygan, WI',
       'United States', 'Bothell, WA', 'Lincoln, NE', 'Overland Park, KS',
       'Santa Monica, CA', 'Portsmouth, NH', 'Ewing, NJ',
       'South San Francisco, CA', 'Palo Alto, CA', 'Bellevue, WA',
       'New Orleans, LA', 'Akron, OH', 'Fort Wayne, IN', 'Woburn, MA',
       'Carson, CA', 'Coral Gables, FL', 'Santa Clara, CA',
       'Brisbane, CA', 'Winter Park, FL', 'Redwood City, CA',
       'Peoria, IL', 'Ipswich, MA', 'Carmel, IN', 'Emeryville, CA',
       'Gaithersburg, MD', 'Longmont, CO', 'Austin, TX', 'Yakima, WA',
 

In [671]:
def job_location_new_feature(value):
    """
    Cleans the input string by removing the newline character ('\n') and any text that follows it.

    Parameters:
    --------------
    value (str): 
        A string containing company names and additional information after a newline character.

    Returns:
    --------------
    str: 
        A cleaned version of the input string with the part after the newline removed.
    """
    state_and_abr =  value.split(', ')
    if len(state_and_abr) > 1:
        if len(state_and_abr) == 2: 
            return state_and_abr[1]
        else:
            return value[:2].upper()
    else:
        return value[:2].upper()

In [672]:
cleaning_df['job_location'] = cleaning_df['Location'].apply(job_location_new_feature)

In [673]:
cleaning_df['job_location'].unique()

array(['NY', 'VA', 'MA', 'CA', 'IL', 'MO', 'WA', 'DC', 'RE', 'TN', 'TX',
       'PA', 'AZ', 'WI', 'GA', 'FL', 'UN', 'NE', 'KS', 'NH', 'NJ', 'LA',
       'OH', 'IN', 'MD', 'CO', 'UT', 'OR', 'MI', 'SC', 'MS', 'AL', 'RI',
       'IA', 'MN', 'OK', 'CT', 'NC', 'TE', 'DE', 'WV'], dtype=object)

In [674]:
def clean_location_faeture(value):
    """
    Extracts the state abbreviation from a given location string. If the location contains a comma, it assumes 
    the format is 'City, State' and returns the state abbreviation. If the location does not contain a comma,
    it returns the first two characters of the string as the state abbreviation (in uppercase).

    Parameters:
    -------------
    value (str): 
        A location string, either in the format 'City, State' or a plain state name.

    Returns:
    -------------
    str: 
        The state abbreviation in uppercase. If the input is a city with a state, the state abbreviation
        is returned. If the input is a state, the first two letters of the state are returned.
    """
    state_and_abr = value.split(', ')
    return state_and_abr[0]

In [675]:
cleaning_df['Location'] = cleaning_df['Location'].apply(clean_location_faeture)

In [676]:
cleaning_df['Location'].unique()

array(['New York', 'Chantilly', 'Boston', 'Newton', 'Santa Barbara',
       'Cambridge', 'Bedford', 'San Diego', 'Chicago', 'Herndon',
       'Saint Louis', 'Richland', 'Northbrook', 'Washington', 'Remote',
       'Memphis', 'Plano', 'West Grove', 'Phoenix', 'Appleton', 'Atlanta',
       'Orlando', 'Lexington', 'McLean', 'San Francisco', 'Sheboygan',
       'United States', 'Bothell', 'Lincoln', 'Overland Park',
       'Santa Monica', 'Portsmouth', 'Ewing', 'South San Francisco',
       'Palo Alto', 'Bellevue', 'New Orleans', 'Akron', 'Fort Wayne',
       'Woburn', 'Carson', 'Coral Gables', 'Santa Clara', 'Brisbane',
       'Winter Park', 'Redwood City', 'Peoria', 'Ipswich', 'Carmel',
       'Emeryville', 'Gaithersburg', 'Longmont', 'Austin', 'Yakima',
       'Santa Cruz', 'Springfield', 'Alexandria', 'Utah', 'Reston',
       'Denver', 'New Jersey', 'Aurora', 'Hill AFB', 'Chandler',
       'Indianapolis', 'Nashville', 'Timonium', 'Burlingame',
       'Annapolis Junction', 'Bethesda', '

In [677]:
cleaning_df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'job_location'],
      dtype='object')

## `Headquarters Feature`

The Headquarters feature contains the location of company headquarters, represented as city and state (or country) pairs, with values such as "New York, NY," "Herndon, VA," and "Santa Barbara, CA." After analyzing the data, it was decided that no modifications or cleaning were necessary for this feature, as the values are already in a standardized format. Therefore, the Headquarters column remains unchanged and is retained as it is for further analysis or processing within the project.

In [678]:
cleaning_df['Headquarters'].unique()

array(['New York, NY', 'Herndon, VA', 'Boston, MA',
       'Bad Ragaz, Switzerland', 'Santa Barbara, CA',
       'Basel, Switzerland', 'Bedford, MA', 'Mountain View, CA',
       'Chicago, IL', 'Mc Lean, VA', 'Saint Louis, MO', 'Richland, WA',
       'Northbrook, IL', 'Princeton, NJ', 'Mays Landing, NJ',
       'Washington, DC', 'Memphis, TN', 'Tempe, AZ', 'Reading, PA',
       'San Francisco, CA', 'Menlo Park, CA', 'Atlanta, GA',
       'Orlando, FL', 'Lexington, MA', 'Falls Church, VA',
       'Sheboygan, WI', 'Seattle, WA', 'Bellevue, WA', 'Lincoln, NE',
       'Chantilly, VA', 'Santa Monica, CA', 'Ewing, NJ',
       'South San Francisco, CA', 'Palo Alto, CA', 'Singapore, Singapore',
       'Cambridge, MA', 'OSAKA, Japan', 'Santa Clara, CA', 'Vienna, VA',
       'New Orleans, LA', 'Akron, OH', 'Zurich, Switzerland',
       'Woburn, MA', 'Carson, CA', 'Coral Gables, FL', 'San Ramon, CA',
       'Brisbane, CA', 'Winter Park, FL', 'San Rafael, CA',
       'Deerfield, IL', 'Ipswich, MA',

## `Size Feature`

The Size feature in the dataset represents the number of employees in a company, with values like '1001 to 5000 employees', '5001 to 10000 employees', and 'Unknown'. To standardize these values, the function convert_size_feature was applied to convert the employee size descriptions into a unified range format, such as '1001-5000', '5001-10000', and '10000+' for companies with more than 10,000 employees. Values like '-1' or 'Unknown' were preserved as 'Unknown'. Finally, the column was renamed to company_employees to better reflect its content.

In [679]:
cleaning_df['Size'].unique()

array(['1001 to 5000 employees', '5001 to 10000 employees',
       '501 to 1000 employees', '51 to 200 employees', '10000+ employees',
       '201 to 500 employees', '1 to 50 employees', '-1', 'Unknown'],
      dtype=object)

In [680]:
def convert_size_feature(value):
    """
    Converts employee size descriptions into a standardized range.

    If the value is '-1' or 'Unknown', it returns 'Unknown'.
    For other values, it extracts the employee range from the description and returns it in a 
    standardized format.

    Parameters:
    ------------
    value (str): 
        A string describing the size of the company in terms of employee count, such as
        '1001 to 5000 employees' or '5001 to 10000 employees'.

    Returns:
    ------------
    str: 
        A standardized employee range, such as '1001-5000'. If the value is '-1' or 'Unknown',
        it returns 'Unknown'.

    """
    if value in ['-1', 'Unknown']:
        return 'Unknown'
    if '10000+' in value:
        return '10000+'
    match = re.search(r'(\d+)(?: to )(\d+)', value)
    return f'{match.group(1)}-{match.group(2)}'

In [681]:
cleaning_df['Size'] = cleaning_df['Size'].apply(convert_size_feature)

In [682]:
cleaning_df['Size'].unique()

array(['1001-5000', '5001-10000', '501-1000', '51-200', '10000+',
       '201-500', '1-50', 'Unknown'], dtype=object)

In [683]:
cleaning_df = cleaning_df.rename(columns={'Size': 'company_employees'})

In [684]:
cleaning_df.head(1)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,company_employees,Founded,Type of ownership,Industry,Sector,Revenue,Competitors,job_location
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,1993,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",NY


## `Founded Feature`

The Founded feature represents the year a company was established, but it contains a significant number of missing values, with 118 entries marked as -1. Given that this column has many missing or invalid entries, and considering it may not be crucial for the analysis, the decision was made to drop this column from the dataset. This will help streamline the data and avoid any potential noise caused by the incomplete information.

In [685]:
cleaning_df['Founded'].unique()

array([1993, 1968, 1981, 2000, 1998, 2010, 1996, 1990, 1983, 2014, 2012,
       2016, 1965, 1973, 1986, 1997, 2015, 1945, 1988, 2017, 2011, 1967,
       1860, 1992, 2003, 1951, 2005, 2019, 1925, 2008, 1999, 1978, 1966,
       1912, 1958, 2013, 1849, 1781, 1926, 2006, 1994, 1863, 1995,   -1,
       1982, 1974, 2001, 1985, 1913, 1971, 1911, 2009, 1959, 2007, 1939,
       2002, 1961, 1963, 1969, 1946, 1957, 1953, 1948, 1850, 1851, 2004,
       1976, 1918, 1954, 1947, 1955, 2018, 1937, 1917, 1935, 1929, 1820,
       1952, 1932, 1894, 1960, 1788, 1830, 1984, 1933, 1880, 1887, 1970,
       1942, 1980, 1989, 1908, 1853, 1875, 1914, 1898, 1956, 1977, 1987,
       1896, 1972, 1949, 1962], dtype=int64)

In [686]:
len(cleaning_df[cleaning_df['Founded'] == -1])

118

In [687]:
cleaning_df = cleaning_df.drop(labels='Founded', axis=1)

In [688]:
cleaning_df.head(1)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,company_employees,Type of ownership,Industry,Sector,Revenue,Competitors,job_location
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,"EmblemHealth, UnitedHealth Group, Aetna",NY


## `Type of ownership Feature`

The Type of ownership feature categorizes companies based on their ownership structure. However, the dataset contains missing or unknown values, represented as -1 and Unknown. To standardize this feature, all -1 values have been converted to 'Unknown'. This ensures consistency in the dataset while preserving relevant ownership classifications such as 'Company - Public', 'Company - Private', 'Government', and others.

In [689]:
cleaning_df['Type of ownership'].unique()

array(['Nonprofit Organization', 'Company - Public',
       'Private Practice / Firm', 'Company - Private', 'Government',
       'Subsidiary or Business Segment', 'Other Organization', '-1',
       'Unknown', 'Hospital', 'Self-employed', 'College / University',
       'Contract'], dtype=object)

In [690]:
def clean_type_of_ownership_feature(value):
    """
    Cleans the 'Type of Ownership' feature by replacing '-1' with 'Unknown'.

    Parameters:
    ------------
    value (str): 
        A string representing the type of ownership.

    Returns:
    ------------
    str: 
        'Unknown' if the input is '-1', otherwise returns the original value.
    """
    if value == '-1':
        return 'Unknown'
    else:
        return value

In [691]:
cleaning_df['Type of ownership'] = cleaning_df['Type of ownership'].apply(clean_type_of_ownership_feature)

In [692]:
cleaning_df['Type of ownership'].unique()

array(['Nonprofit Organization', 'Company - Public',
       'Private Practice / Firm', 'Company - Private', 'Government',
       'Subsidiary or Business Segment', 'Other Organization', 'Unknown',
       'Hospital', 'Self-employed', 'College / University', 'Contract'],
      dtype=object)

In [693]:
cleaning_df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'company_employees',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'job_location'],
      dtype='object')

## `Industry Feature`

The Industry feature classifies companies based on their sector of operation. However, some entries in the dataset are missing or labeled as -1. To maintain data consistency, all -1 values have been converted to 'Unknown'. This ensures that the dataset remains structured while retaining valuable industry classifications such as 'Insurance Carriers', 'Biotech & Pharmaceuticals', 'IT Services', and others.

In [694]:
cleaning_df['Industry'].unique()

array(['Insurance Carriers', 'Research & Development', 'Consulting',
       'Electrical & Electronic Manufacturing', 'Advertising & Marketing',
       'Computer Hardware & Software', 'Biotech & Pharmaceuticals',
       'Consumer Electronics & Appliances Stores',
       'Enterprise Software & Network Solutions', 'IT Services', 'Energy',
       'Chemical Manufacturing', 'Federal Agencies', 'Internet',
       'Health Care Services & Hospitals',
       'Investment Banking & Asset Management', 'Aerospace & Defense',
       'Utilities', '-1', 'Express Delivery Services',
       'Staffing & Outsourcing', 'Insurance Agencies & Brokerages',
       'Consumer Products Manufacturing', 'Industrial Manufacturing',
       'Food & Beverage Manufacturing', 'Banks & Credit Unions',
       'Video Games', 'Shipping', 'Telecommunications Services',
       'Lending', 'Cable, Internet & Telephone Providers', 'Real Estate',
       'Venture Capital & Private Equity', 'Miscellaneous Manufacturing',
       'Oil 

In [695]:
def clean_industry_feature(value):
    """
    Cleans the 'Industry' feature by replacing '-1' with 'Unknown'.

    Parameters:
    ------------
    value (str): 
        A string representing the industry.

    Returns:
    ------------
    str: 
        'Unknown' if the input is '-1', otherwise returns the original value.
    """
    if value == '-1':
        return 'Unknown'
    else:
        return value

In [696]:
cleaning_df['Industry'] = cleaning_df['Industry'].apply(clean_industry_feature)

In [697]:
cleaning_df['Industry'].unique()

array(['Insurance Carriers', 'Research & Development', 'Consulting',
       'Electrical & Electronic Manufacturing', 'Advertising & Marketing',
       'Computer Hardware & Software', 'Biotech & Pharmaceuticals',
       'Consumer Electronics & Appliances Stores',
       'Enterprise Software & Network Solutions', 'IT Services', 'Energy',
       'Chemical Manufacturing', 'Federal Agencies', 'Internet',
       'Health Care Services & Hospitals',
       'Investment Banking & Asset Management', 'Aerospace & Defense',
       'Utilities', 'Unknown', 'Express Delivery Services',
       'Staffing & Outsourcing', 'Insurance Agencies & Brokerages',
       'Consumer Products Manufacturing', 'Industrial Manufacturing',
       'Food & Beverage Manufacturing', 'Banks & Credit Unions',
       'Video Games', 'Shipping', 'Telecommunications Services',
       'Lending', 'Cable, Internet & Telephone Providers', 'Real Estate',
       'Venture Capital & Private Equity', 'Miscellaneous Manufacturing',
       

## `Sector Feature`

The Sector feature categorizes companies into broader industries, such as 'Information Technology', 'Health Care', and 'Finance'. Since some entries in the dataset were labeled as -1, these have been replaced with 'Unknown' to ensure data consistency. This transformation helps maintain a structured dataset while preserving meaningful sector classifications.

In [698]:
cleaning_df['Sector'].unique()

array(['Insurance', 'Business Services', 'Manufacturing',
       'Information Technology', 'Biotech & Pharmaceuticals', 'Retail',
       'Oil, Gas, Energy & Utilities', 'Government', 'Health Care',
       'Finance', 'Aerospace & Defense', '-1',
       'Transportation & Logistics', 'Media', 'Telecommunications',
       'Real Estate', 'Travel & Tourism', 'Agriculture & Forestry',
       'Education', 'Accounting & Legal', 'Non-Profit',
       'Construction, Repair & Maintenance', 'Consumer Services'],
      dtype=object)

In [699]:
def clean_sector_feature(value):
    """
    Cleans the 'Sector' feature by replacing '-1' with 'Unknown'.

    Parameters:
    ------------
    value (str): 
        A string representing the industry.

    Returns:
    ------------
    str: 
        'Unknown' if the input is '-1', otherwise returns the original value.
    """
    if value == '-1':
        return 'Unknown'
    else:
        return value

In [700]:
cleaning_df['Sector'] = cleaning_df['Sector'].apply(clean_sector_feature)

## `Revenue Feature`

In [701]:
cleaning_df['Revenue'].unique()

array(['Unknown / Non-Applicable', '$1 to $2 billion (USD)',
       '$100 to $500 million (USD)', '$10+ billion (USD)',
       '$2 to $5 billion (USD)', '$500 million to $1 billion (USD)',
       '$5 to $10 billion (USD)', '$10 to $25 million (USD)',
       '$25 to $50 million (USD)', '$50 to $100 million (USD)',
       '$1 to $5 million (USD)', '$5 to $10 million (USD)',
       'Less than $1 million (USD)', '-1'], dtype=object)

In [702]:
def clean_revenue_feature(value):
    """
    Cleans the 'Revenue' feature by replacing '-1' with 'Unknown / Non-Applicable'.

    Parameters:
    ------------
    value (str): 
        A string representing the industry.

    Returns:
    ------------
    str: 
        'Unknown' if the input is '-1', otherwise returns the original value.
    """
    if value == '-1':
        return 'Unknown / Non-Applicable'
    else:
        return value

In [703]:
cleaning_df['Revenue'] = cleaning_df['Revenue'].apply(clean_revenue_feature)

In [704]:
cleaning_df['Revenue'].unique()

array(['Unknown / Non-Applicable', '$1 to $2 billion (USD)',
       '$100 to $500 million (USD)', '$10+ billion (USD)',
       '$2 to $5 billion (USD)', '$500 million to $1 billion (USD)',
       '$5 to $10 billion (USD)', '$10 to $25 million (USD)',
       '$25 to $50 million (USD)', '$50 to $100 million (USD)',
       '$1 to $5 million (USD)', '$5 to $10 million (USD)',
       'Less than $1 million (USD)'], dtype=object)

In [705]:
cleaning_df.columns

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'company_employees',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'job_location'],
      dtype='object')

## `Competitors Feature`

The Competitors feature was dropped because it contained a large number of missing values (501 entries labeled as -1). Given its limited usefulness for analysis and the high percentage of missing data, removing the column helps streamline the dataset.

In [706]:
cleaning_df['Competitors'].unique()

array(['EmblemHealth, UnitedHealth Group, Aetna', '-1',
       'MKS Instruments, Pfeiffer Vacuum, Agilent Technologies',
       'Commerce Signals, Cardlytics, Yodlee',
       'Square, PayPal, H&R Block',
       'Leidos, CACI International, Booz Allen Hamilton',
       'Slalom, Daugherty Business Solutions',
       'Oak Ridge National Laboratory, National Renewable Energy Lab, Los Alamos National Laboratory',
       'CDW, PCM, SHI International',
       'Crossix Solutions Inc., AppNexus, The Trade Desk',
       'Northwestern Mutual', 'Puppet, Ansible, SaltStack',
       'Enlivant, Sunrise Senior Living, Brookdale Senior Living',
       'TrueCar, Cars.com, Kelley Blue Book',
       'Travelers, Allstate, State Farm', 'Novartis, Baxter, Pfizer',
       'Skyhigh Networks, Zscaler, NortonLifeLock',
       'Facebook, Google, Pinterest', 'DoorDash, Uber, Grubhub',
       'Munich Re, Hannover RE, SCOR', "IMAGE Skincare, Aveda, Kiehl's",
       'Luxoft, EPAM, Capgemini Invent', 'Sequenom',
     

In [707]:
len(cleaning_df[cleaning_df['Competitors'] == '-1'])

501

In [708]:
cleaning_df = cleaning_df.drop(labels='Competitors', axis=1)

In [709]:
cleaning_df.head(1)

,Job Title,Salary Estimate,Job Description,Rating,Company Name,Location,Headquarters,company_employees,Type of ownership,Industry,Sector,Revenue,job_location
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,NY


# 4 Features

## 4.1 Rename Features

The feature names were standardized by converting them to lowercase and replacing spaces with underscores. This improves consistency and makes it easier to reference column names in code. 

In [710]:
features_cleaning_df = cleaning_df.copy()

In [711]:
features_cleaning_df.columns = features_cleaning_df.columns.str.lower().str.replace(' ', '_')

In [712]:
features_cleaning_df.columns

Index(['job_title', 'salary_estimate', 'job_description', 'rating',
       'company_name', 'location', 'headquarters', 'company_employees',
       'type_of_ownership', 'industry', 'sector', 'revenue', 'job_location'],
      dtype='object')

In [713]:
features_cleaning_df.head(1)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,company_employees,type_of_ownership,industry,sector,revenue,job_location
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,NY


## 4.2 Min Salary and Max Salary

The salary_estimate column was successfully split into min_salary and max_salary, providing clearer salary range insights. These two new columns allow for more granular salary analysis, such as calculating salary averages or filtering jobs by salary range.

In [714]:
features_cleaning_df['salary_estimate'].unique()

array(['137-171', '75-131', '79-131', '99-132', '90-109', '101-165',
       '56-97', '79-106', '71-123', '90-124', '91-150', '141-225',
       '145-225', '79-147', '122-146', '112-116', '110-163', '124-198',
       '79-133', '69-116', '31-56', '95-119', '212-331', '66-112',
       '128-201', '138-158', '80-132', '87-141', '92-155', '105-167'],
      dtype=object)

In [715]:
features_cleaning_df['min_salary'] = features_cleaning_df['salary_estimate'].apply(lambda x: x.split('-')[0])

In [716]:
features_cleaning_df['min_salary'] = features_cleaning_df['min_salary'].astype(int)

In [717]:
features_cleaning_df['min_salary'].unique()

array([137,  75,  79,  99,  90, 101,  56,  71,  91, 141, 145, 122, 112,
       110, 124,  69,  31,  95, 212,  66, 128, 138,  80,  87,  92, 105])

In [718]:
features_cleaning_df['max_salary'] = features_cleaning_df['salary_estimate'].apply(lambda x: x.split('-')[1])

In [719]:
features_cleaning_df['max_salary'] = features_cleaning_df['max_salary'].astype(int)

In [720]:
features_cleaning_df['max_salary'].unique()

array([171, 131, 132, 109, 165,  97, 106, 123, 124, 150, 225, 147, 146,
       116, 163, 198, 133,  56, 119, 331, 112, 201, 158, 141, 155, 167])

In [721]:
features_cleaning_df.head(1)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,company_employees,type_of_ownership,industry,sector,revenue,job_location,min_salary,max_salary
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,NY,137,171


## 4.3 Average Salary New Feature

In this step of data preprocessing, we created a new feature, average_salary, by calculating the average of the min_salary and max_salary for each job listing. This allows for a more straightforward representation of the salary range, enabling us to analyze and compare job salaries on a more consistent basis.

The new average_salary feature was computed by summing the min_salary and max_salary for each entry and dividing the result by 2. We then converted the values to integers to ensure consistency and make the data ready for analysis or model training.

This transformation simplifies the salary information, helping in tasks such as salary distribution analysis, comparisons, and even in prediction models for salary estimations.

In [722]:
features_cleaning_df['average_salary'] = features_cleaning_df[['min_salary', 'max_salary']].apply(lambda x: (x['min_salary'] + x['max_salary']) / 2, axis=1)

In [723]:
features_cleaning_df['average_salary'] = features_cleaning_df['average_salary'].astype(int)

In [724]:
features_cleaning_df['average_salary'].unique()

array([154, 103, 105, 115,  99, 133,  76,  92,  97, 107, 120, 183, 185,
       113, 134, 114, 136, 161, 106,  43, 271,  89, 164, 148, 123])

## 4.4 Same State Feature

In this step, we created a new feature, same_state, to indicate whether the job's location is the same as the company's headquarters location. This binary feature helps identify if a job is located in the same state as the company's headquarters, which could be useful for understanding regional hiring trends or employee relocation preferences.

The same_state feature was computed by comparing the location (job location) with the state portion of the headquarters (company headquarters). If the state in the location matches the state extracted from the headquarters, the same_state feature is assigned a value of 1; otherwise, it is assigned a value of 0.

The result is a binary feature (1 or 0), where 1 indicates the job is in the same state as the company's headquarters, and 0 indicates it is not. This new feature adds additional value for analysis, particularly when exploring whether employees are generally hired locally or across state lines.

In [725]:
features_cleaning_df['same_state'] = features_cleaning_df[['location', 'headquarters']].apply(lambda x: 1 if  x['location'] == x['headquarters'].split(', ')[0] else 0, axis=1)

In [726]:
features_cleaning_df['same_state'].unique()

array([1, 0], dtype=int64)

In [727]:
features_cleaning_df['same_state'].value_counts()

same_state
0    402
1    270
Name: count, dtype: int64

## 4.5 Company Age New Feature

In [728]:
features_cleaning_df['company_age'] = df['Founded'].apply(lambda x: datetime.datetime.now().year - x if x != -1 else -1)

In [729]:
features_cleaning_df['company_age'].unique()

array([ 32,  57,  44,  25,  27,  15,  29,  35,  42,  11,  13,   9,  60,
        52,  39,  28,  10,  80,  37,   8,  14,  58, 165,  33,  22,  74,
        20,   6, 100,  17,  26,  47,  59, 113,  67,  12, 176, 244,  99,
        19,  31, 162,  30,  -1,  43,  51,  24,  40, 112,  54, 114,  16,
        66,  18,  86,  23,  64,  62,  56,  79,  68,  72,  77, 175, 174,
        21,  49, 107,  71,  78,  70,   7,  88, 108,  90,  96, 205,  73,
        93, 131,  65, 237, 195,  41,  92, 145, 138,  55,  83,  45,  36,
       117, 172, 150, 111, 127,  69,  48,  38, 129,  53,  76,  63],
      dtype=int64)

In [730]:
features_cleaning_df['company_age'].head(3)

0    32
1    57
2    44
Name: company_age, dtype: int64

In [731]:
features_cleaning_df.head(3)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,company_employees,type_of_ownership,industry,sector,revenue,job_location,min_salary,max_salary,average_salary,same_state,company_age
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,Insurance,Unknown / Non-Applicable,NY,137,171,154,1,32
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,Chantilly,"Herndon, VA",5001-10000,Company - Public,Research & Development,Business Services,$1 to $2 billion (USD),VA,137,171,154,0,57
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,Boston,"Boston, MA",1001-5000,Private Practice / Firm,Consulting,Business Services,$100 to $500 million (USD),MA,137,171,154,1,44


## 4.6 Python, Excel, Hadoop, Spark, AWS, Tableau, and Big Data New Features

In this step, we created new features for specific skills mentioned in the job descriptions, such as Python, Excel, Hadoop, Spark, AWS, Tableau, and Big Data. These features were added to the dataset to provide a better understanding of the technical skills required for each job.

Here’s how it works:

1. For each skill in the list (python, excel, hadoop, spark, aws, tableau, big data), a new binary feature was created. The value for each new feature is determined by whether the corresponding skill appears in the job description text.

2. The job description is converted to lowercase to ensure the skill is detected regardless of case sensitivity.

3. If the skill is found in the job description, the corresponding feature is set to 1 (indicating that the skill is required for the job). If the skill is not found, the feature is set to 0.

4. The feature name big data was renamed to big_data to follow a consistent naming convention, replacing the space with an underscore.

These newly created features will allow us to quickly assess which jobs require specific technical skills, and they are especially helpful for machine learning models or further analysis of job trends in relation to specific skills.

In [732]:
features = ['python', 'excel', 'hadoop', 'spark', 'aws', 'tableau', 'big data']

for feature in features:
    features_cleaning_df[feature] = features_cleaning_df['job_description'].apply(lambda x: 1 if feature in x.lower() else 0)

In [733]:
features_cleaning_df = features_cleaning_df.rename(columns={'big data': 'big_data'})

In [734]:
features_cleaning_df.head(1)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,company_employees,type_of_ownership,industry,...,average_salary,same_state,company_age,python,excel,hadoop,spark,aws,tableau,big_data
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,...,154,1,32,0,0,0,0,1,0,0


## Job Simp Feature

In this step, we create a new feature called job_simp, which simplifies job titles into broader categories. Here's a breakdown of how this works:

1. Job Title Classification:
* The function create_job_simp_feature checks for specific keywords (e.g., "director", "manager", "data scientist") in the job title. If one of these keywords is found in the job title, the function returns a simplified label corresponding to the role, such as 'Director', 'Manager', 'Data Scientist', etc. If none of the predefined keywords is found, it returns 'Na' (not available), indicating that the job title doesn't match any of the specified categories.

2. Use of .apply():
* The function is applied to the job_title column using .apply() to create the new job_simp feature.

This categorization is helpful because it groups job titles into broad, simplified categories, making it easier to analyze job trends and focus on specific types of roles, rather than dealing with the variety of job titles that may exist in the dataset. 

In [743]:
features_cleaning_df['job_title'].unique()

array(['Sr Data Scientist', 'Data Scientist',
       'Data Scientist / Machine Learning Expert',
       'Staff Data Scientist - Analytics',
       'Data Scientist - Statistics, Early Career', 'Data Modeler',
       'Experienced Data Scientist', 'Data Scientist - Contract',
       'Data Analyst II', 'Medical Lab Scientist',
       'Data Scientist/Machine Learning', 'Human Factors Scientist',
       'Business Intelligence Analyst I- Data Insights',
       'Data Scientist - Risk', 'Data Scientist-Human Resources',
       'Senior Research Statistician- Data Scientist', 'Data Engineer',
       'Associate Data Scientist', 'Business Intelligence Analyst',
       'Senior Analyst/Data Scientist', 'Data Analyst',
       'Machine Learning Engineer', 'Data Analyst I',
       'Scientist - Molecular Biology',
       'Computational Scientist, Machine Learning',
       'Senior Data Scientist', 'Jr. Data Engineer',
       'E-Commerce Data Analyst', 'Data Analytics Engineer',
       'Product Data Scient

In [736]:
def create_job_simp_feature(value):
    if 'director' in value.lower():
        return 'Director'
    elif 'manager' in value.lower():
        return 'Manager'
    elif 'data scientist' in value.lower():
        return 'Data Scientist'
    elif 'analyst' in value.lower():
        return 'Analyst'
    elif 'data engineer' in value.lower():
        return 'Data Engineer'
    elif 'machine learning' in value.lower():
        return 'Machine Learning'
    else:
        return 'Na'
    
    

In [737]:
features_cleaning_df['job_simp'] = features_cleaning_df['job_title'].apply(create_job_simp_feature)

In [742]:
features_cleaning_df[['job_title', 'job_simp']]

,job_title,job_simp
0,Sr Data Scientist,Data Scientist
1,Data Scientist,Data Scientist
2,Data Scientist,Data Scientist
3,Data Scientist,Data Scientist
4,Data Scientist,Data Scientist
...,...,...
667,Data Scientist,Data Scientist
668,Data Scientist,Data Scientist
669,Data Scientist,Data Scientist
670,Data Scientist,Data Scientist


## Seniority New Feature

In this part of the project, I focused on creating a new feature called seniority to classify job titles based on their seniority level. The goal was to categorize roles as 'Junior', 'Senior', or 'Na' (not available) based on specific keywords in the job title.

In [758]:
def create_seniority_feature(value):
    if 'jr' in value.lower():
        return 'Junior'
    if 'sr' in value.lower() or 'senior' in value.lower() or 'principal' in value.lower() or 'lead' in value.lower() or 'expert' in value.lower():
        return 'Senior'
    else:
        return 'Na'

In [759]:
features_cleaning_df['seniority'] = features_cleaning_df['job_title'].apply(create_seniority_feature)

In [760]:
features_cleaning_df.head()

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,company_employees,type_of_ownership,industry,...,company_age,python,excel,hadoop,spark,aws,tableau,big_data,job_simp,seniority
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,...,32,0,0,0,0,1,0,0,Data Scientist,Senior
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,Chantilly,"Herndon, VA",5001-10000,Company - Public,Research & Development,...,57,0,0,1,0,0,0,1,Data Scientist,Na
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,Boston,"Boston, MA",1001-5000,Private Practice / Firm,Consulting,...,44,1,1,0,0,1,0,0,Data Scientist,Na
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,Newton,"Bad Ragaz, Switzerland",501-1000,Company - Public,Electrical & Electronic Manufacturing,...,25,1,1,0,0,1,0,0,Data Scientist,Na
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,New York,"New York, NY",51-200,Company - Private,Advertising & Marketing,...,27,1,1,0,0,0,0,0,Data Scientist,Na


# 5. Outliers

In [765]:
outliers = delete_outliers(features_cleaning_df, 2, [x for x in features_cleaning_df.columns if features_cleaning_df[x].dtype != 'object'])

In [766]:
len(outliers)

46

In [768]:
features_cleaning_df = features_cleaning_df.drop(outliers, axis=0).reset_index(drop=True)

In [769]:
features_cleaning_df.to_csv('finish.csv')

In [771]:
finished_df = features_cleaning_df.copy()

In [772]:
finished_df.head(10)

,job_title,salary_estimate,job_description,rating,company_name,location,headquarters,company_employees,type_of_ownership,industry,...,company_age,python,excel,hadoop,spark,aws,tableau,big_data,job_simp,seniority
0,Sr Data Scientist,137-171,Description\n\nThe Senior Data Scientist is re...,3.1,Healthfirst,New York,"New York, NY",1001-5000,Nonprofit Organization,Insurance Carriers,...,32,0,0,0,0,1,0,0,Data Scientist,Senior
1,Data Scientist,137-171,"Secure our Nation, Ignite your Future\n\nJoin ...",4.2,ManTech,Chantilly,"Herndon, VA",5001-10000,Company - Public,Research & Development,...,57,0,0,1,0,0,0,1,Data Scientist,Na
2,Data Scientist,137-171,Overview\n\n\nAnalysis Group is one of the lar...,3.8,Analysis Group,Boston,"Boston, MA",1001-5000,Private Practice / Firm,Consulting,...,44,1,1,0,0,1,0,0,Data Scientist,Na
3,Data Scientist,137-171,JOB DESCRIPTION:\n\nDo you have a passion for ...,3.5,INFICON,Newton,"Bad Ragaz, Switzerland",501-1000,Company - Public,Electrical & Electronic Manufacturing,...,25,1,1,0,0,1,0,0,Data Scientist,Na
4,Data Scientist,137-171,Data Scientist\nAffinity Solutions / Marketing...,2.9,Affinity Solutions,New York,"New York, NY",51-200,Company - Private,Advertising & Marketing,...,27,1,1,0,0,0,0,0,Data Scientist,Na
5,Data Scientist,137-171,About Us:\n\nHeadquartered in beautiful Santa ...,4.2,HG Insights,Santa Barbara,"Santa Barbara, CA",51-200,Company - Private,Computer Hardware & Software,...,15,1,1,1,1,0,0,0,Data Scientist,Na
6,Data Scientist / Machine Learning Expert,137-171,Posting Title\nData Scientist / Machine Learni...,3.9,Novartis,Cambridge,"Basel, Switzerland",10000+,Company - Public,Biotech & Pharmaceuticals,...,29,1,0,0,0,0,0,0,Data Scientist,Senior
7,Data Scientist,137-171,Introduction\n\nHave you always wanted to run ...,3.5,iRobot,Bedford,"Bedford, MA",1001-5000,Company - Public,Consumer Electronics & Appliances Stores,...,35,1,0,0,0,0,0,0,Data Scientist,Na
8,Staff Data Scientist - Analytics,137-171,Intuit is seeking a Staff Data Scientist to co...,4.4,Intuit - Data,San Diego,"Mountain View, CA",5001-10000,Company - Public,Computer Hardware & Software,...,42,0,0,0,0,0,0,0,Data Scientist,Na
9,Data Scientist,137-171,Ready to write the best chapter of your career...,3.6,XSELL Technologies,Chicago,"Chicago, IL",51-200,Company - Private,Enterprise Software & Network Solutions,...,11,1,0,0,0,0,0,0,Data Scientist,Na
